In [2]:
import tensorflow as tf

In [6]:
class Inception:

  def __init__(self,input_shape,classes=1000):

    self.inputs = tf.keras.Input(shape=input_shape)

    conv_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(7,7),strides=2,padding='same')(self.inputs)
    pool_1 = tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=2,padding='same')(conv_1)
  
    conv_2_reduce = tf.keras.layers.Conv2D(filters=64,kernel_size=(1,1))(pool_1)
    conv_2 = tf.keras.layers.Conv2D(filters=192, kernel_size=(3,3),strides=1,padding='same')(conv_2_reduce)

    pool_2 = tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=2,padding='same')(conv_2)

    # Inception 3a
    inception_3a =self.inception_module(pool_2,64,96,128,16,32,32)
  
    # Inception 3b
    inception_3b =self.inception_module(inception_3a,128,128,192,32,96,64)
  
    pool_3 = tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=2,padding='same')(inception_3b)
  
    # Inception 4a
    inception_4a = self.inception_module(pool_3,192,96,208,16,48,64)
  
    # Auxilary Classifier 4a
    aux1 = tf.keras.layers.AveragePooling2D((5,5),strides=3)(inception_4a)
    aux1 =  tf.keras.layers.Conv2D(filters=128,kernel_size=(1,1),activation='relu')(aux1)
    aux1 = tf.keras.layers.Flatten()(aux1)
    aux1 = tf.keras.layers.Dense(1024,activation='relu')(aux1)
    aux1 = tf.keras.layers.Dropout(0.7)(aux1)
    aux1 = tf.keras.layers.Dense(classes,activation='softmax')(aux1)

    # Inception 4b
    inception_4b = self.inception_module(inception_4a,160,112,224,24,64,64)
  
    # Inception 4c
    
    inception_4c = self.inception_module(inception_4b,128,128,256,24,64,64)
  
    # Inception 4d
    
    inception_4d = self.inception_module(inception_4c,112,144,288,32,64,64)
  
    # Auxilary layer 4d
    
    aux2 = tf.keras.layers.AveragePooling2D((5,5),strides=3)(inception_4d)
    aux2 =  tf.keras.layers.Conv2D(filters=128,kernel_size=(1,1),activation='relu')(aux2)
    aux2 = tf.keras.layers.Flatten()(aux2)
    aux2 = tf.keras.layers.Dense(1024,activation='relu')(aux2)
    aux2 = tf.keras.layers.Dropout(0.7)(aux2)
    aux2 = tf.keras.layers.Dense(classes,activation='softmax')(aux2)


    # Inception 4e
    inception_4e = self.inception_module(inception_4d,256,160,320,32,128,128)
  
    # Pool Layer

    pool_4 = tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=2,padding='same')(inception_4e)
 
    # Inception 5a
    inception_5a = self.inception_module(pool_4,256,160,320,32,128,128)
 
    # Inception 5b
    inception_5b = self.inception_module(inception_5a,384,192,384,48,128,128)
 
    # Last Layer

    avg_pool =  tf.keras.layers.AveragePooling2D((7,7),strides=1)(inception_5b)
    drop_out = tf.keras.layers.Dropout(0.4)(avg_pool)
    flatten = tf.keras.layers.Flatten()(drop_out)
    dense = tf.keras.layers.Dense(1000)(drop_out)
    self.out = tf.keras.layers.Dense(classes,activation='softmax')(dense)

    self.model = tf.keras.Model(inputs=self.inputs,outputs=[self.out,aux1,aux2])





  
  def inception_module(self,inputs,
                  filter_1x1,
                  filter_3x3_1x1_reduce,
                  filter_3x3,
                  filter_5x5_1x1_reduce,
                  filter_5x5,
                  proj_pool):
    
    conv_1x1 = tf.keras.layers.Conv2D(filters=filter_1x1, kernel_size=(1,1),strides=1,padding='same')(inputs)

    conv_3x3_reduce =  tf.keras.layers.Conv2D(filters=filter_3x3_1x1_reduce, kernel_size=(1,1),strides=1,padding='same')(inputs)
    conv_3x3_reduce =  tf.keras.layers.Conv2D(filters=filter_3x3, kernel_size=(3,3),strides=1,padding='same')(conv_3x3_reduce)

    conv_5x5_reduce =  tf.keras.layers.Conv2D(filters=filter_5x5_1x1_reduce, kernel_size=(1,1),strides=1,padding='same')(inputs)
    conv_5x5_reduce =  tf.keras.layers.Conv2D(filters=filter_5x5, kernel_size=(5,5),strides=1,padding='same')(conv_5x5_reduce)

    proj_layer = tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=1,padding='same')(inputs)
    proj_layer =  tf.keras.layers.Conv2D(filters=proj_pool, kernel_size=(1,1),strides=1,padding='same')(proj_layer)

    return tf.keras.layers.concatenate([conv_1x1, conv_3x3_reduce, conv_5x5_reduce,proj_layer], axis=3)


  def get_model(self):
    return self.model


In [7]:
model = Inception((224,224,3)).get_model()

In [8]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_118 (Conv2D)            (None, 112, 112, 64  9472        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_26 (MaxPooling2D  (None, 56, 56, 64)  0           ['conv2d_118[0][0]']             
 )                                                                                          